# Advanced Speed and Accuracy Topics

This is going to be a bird's eye view of some of the advanced topics in Julia, particularly focusing on issues of speed and performance.

Julia is built with a number of components which enable it to obtain massive performance, despite its scripted nature.
You've already seen 
- multiple dispatch
- types, when necessary

We are skipping the built-in parallelism. This is for the very good reason that, even though it's designed-in from the outset of the language the syntax is currently being stabilised and should be final from the next release (_the disadvantage of being an early adopter_).

## Calling a C function natively

We can call **C** and **Fortran** functions natively. That means that you can potentially have an ultra high-performance library, which has been worked on for years, and you get to access its functions directly from Julia at **0** computational overhead.

>ccall((:function_name, "library_name"), return_type, (argument_types), argument_values)

Current requirements: the library must be *dynamically* linkable (not static).

Note that, getting the function call syntax correct is typically the biggest stumbling block to working with C and Fortran libraries from Julia. In addition, you don't have the benefit of interfacing via header files so the handling of returned values can be tricky if they're not what you were expecting.

Also, for Fortran users, you should note that Fortran compilers often change the case of a function call. From Julia you need to call the *compiled* version of the function call, not the original source code spelling.

In [ ]:
@show ccall((:clock,),Int64,())
@show ccall((:clock,),Int64,())
@show ccall((:clock,),Int64,())

In [ ]:
@show ccall((:rand,), Int64, ())
@show ccall((:rand,), Int64, ())
@show ccall((:rand,), Int64, ())

In [ ]:
ccall((:printf,), Int64, (Cstring,), "blah")

In [ ]:
ccall((:printf,), Int64, (Cstring,Int64), "blah %d \n", 999)

In [ ]:
function getenv(var::AbstractString)
    val = ccall((:getenv, "libc"),
                Cstring, (Cstring,), var)
    if val == C_NULL
        error("getenv: undefined variable: ", var)
    end
    unsafe_string(val)
end

In [ ]:
getenv("PATH")

In [ ]:
getenv("THIS_value_DOESNT_exist")

In practice, you may never need to use this functionality depending on your position on the power-user spectrum.

But even if you're not using it directly, under the hood this is how many of the Packages are interfacing with numerical libraries which have been developed and optimised in C and Fortran since the 70's.

https://docs.julialang.org/en/stable/manual/calling-c-and-fortran-code/

## OpenCL

I use OpenCL extensively in my work. You may be familiar with CUDA. There is also a CUDAnative package for Julia. The principles I'm going to present are similar for both interfaces.

OpenCL will work out of the box for Mac users. For Windows and Linux users it typically requires some external packages be installed. This is not a workshop on OpenCL, so we'll provide a notebook with the calculated outputs which you can follow, but please note when installing OpenCL that you need a **driver** or a **library** which provides OpenCL support directly for your system and not just the **header** files. (apt-get install opencl typically only grabs the headers)

If you want to install OpenCL (after the workshop) there are a number of options:
1. If you have an NVIDIA GPU then you automatically have OpenCL installed if you install CUDA development system. Note that the NVIDIA implementation can only compile OpenCL for the GPU device.
2. You can install ATIs APP SDK, which is natively OpenCL, for ATI GPU devices. _This system can also compile for CPU giving you the best of both worlds!_
3. Intel provide OpenCL compilers for CPUs.
4. Apple pre-install OpenCL support on their computers, for both CPU and GPU.

In [ ]:
Pkg.add("OpenCL")

In [ ]:
using OpenCL

The writers of the OpenCL package have made extensive use of the ccall( ) approach to directly interface with OpenCL (which is typically accessed from C).

Here I will present a few wrapper functions, which I have written, which most OpenCL developers typically need to write themselves when they begin using the langauge.


In [ ]:
function print_cl_stuff()
    print("Number of Platforms: ", cl.num_platforms(), "\n")
    for plat in cl.num_platforms()
        print("Platform[", plat, "]: ", cl.platforms()[plat], "\n")
        #print("Platforms: ", cl.platforms())
        numdevices = length(cl.devices())
        print("\tNumber of Devices: ", numdevices, "\n")
        for dev = 1:numdevices
            print("\tDevice[", dev, "]: ", cl.devices()[dev], "\n")
        end
        #print("\nDevices: ", cl.devices(), "\n")
    end
end

Here we're making use of the following functions.

> cl.num_platforms( )

> cl.platforms( )[index]

> cl.devices( )

> cl.devices( )[index]

These are **very** lightly wrapped versions of the C functions which are part of the OpenCL API.

In [ ]:
print_cl_stuff()

In OpenCL you may have multiple platforms installed on your system (these are the different implementations mentioned in the install notes above).

When you're setting up a computation, you need to choose:
- platform
- device

and create a memory (plus device)
- context

This latter is basically the namespace within which your computation will operate.

In [ ]:
function set_device(device_id, platform)
    device = cl.devices()[device_id]
    return device
end

function set_device(device_id)
    platform = set_platform()
    return set_device(device_id, platform)
end

function set_context(device)
    ctx = cl.Context(device)
    return ctx
end

function set_platform(platform_id=0)
    platform = cl.Platform(platform_id)
    return platform
end;

In [ ]:
our_platform = set_platform()

In [ ]:
cl.Platform(0)

Setting the platform defaults to the first one (since there is often only one installed platform).

In [ ]:
our_device = set_device(1)

Notice that the indexing of the array here is 1 based, as it's still in Julia. However the platform id above is indexed from 0 as it's a value which is directly passed to the C function.

In [ ]:
cl.devices()[2]

In [ ]:
our_context = set_context(our_device)

We now have access to device 1, via our_context, and we have a handle for our_platform. These are all necessary components for running an OpenCL kernel.

In [ ]:
cl.CLError(-10)

Many functions in OpenCL can return an error code. This has its own format but you can access it via CLError.

In [ ]:
cl.error_description(cl.CLError(-10))

Normally the above line will work. But the version of the package that I was working with had a bug. I've submitted a pull request so it will hopefully work by the time of the workshop. 

In any case, the expected behaviour is to print out a useful string regarding the meaning of any returned error code.

Note that OpenCL does not provide this functionality, this is a feature of the Julia OpenCL package.

## OpenCL showcase example

*based on code written by Thomas McColgan and Rike Schuppner at a workshop in Lychen, 2017*


In [ ]:
diff_kernel = "
inline int xy_to_i(int x, int y, int col_size, int mesh_size) {
    return (x % col_size) + (y % col_size) * col_size; 
}

__kernel void diff(__global const float *a,
__global float *diff,
int col_size,
int mesh_size)
    {
      int gid = get_global_id(0);
    int x_idx = gid % col_size;
    int y_idx = gid / col_size;
    
    diff[gid] = - 4. * a[gid]
    + a[xy_to_i(x_idx - 1, y_idx, col_size, mesh_size)]
    + a[xy_to_i(x_idx + 1, y_idx, col_size, mesh_size)]
    + a[xy_to_i(x_idx, y_idx - 1, col_size, mesh_size)]
    + a[xy_to_i(x_idx, y_idx + 1, col_size, mesh_size)];
    }
";

In [ ]:
sum_kernel = "
   __kernel void sum(__global float *a,
__global const float *diff,
float step_size)
    {
      int gid = get_global_id(0);
    a[gid] += step_size * diff[gid];
    }
";

OpenCL allows us to run **kernels** on the GPU/CPU device.

In our example each parallel computational unit operates its own version of the *kernel* on a different unit in an array.

In [ ]:
function initial_cond(big_array=true)
    if big_array
        a = zeros(Float32, (5000, 5000))
    else
        a = zeros(Float32, (1000, 1000))
    end
    a[Int(size(a)[1] / 2), Int(size(a)[2] / 2)] = 10000
    return a
end;

In [ ]:
function do_openCL(input, device_index=1)
#    device, ctx, queue = cl.create_compute_context()
#    print(device, ctx, queue)
    device = set_device(device_index)
    # on my system device 1 is CPU, device 2 is GPU
    ctx = set_context(device)
    queue  = cl.CmdQueue(ctx)
    print(ctx, device, queue)

    input_buff = cl.Buffer(Float32, ctx, (:rw, :copy), hostbuf=input)
    diff_buff = cl.Buffer(Float32, ctx, :rw, length(input))

    diff_p = cl.Program(ctx, source=diff_kernel) |> cl.build!
    diff_k = cl.Kernel(diff_p, "diff")

    sum_p = cl.Program(ctx, source=sum_kernel) |> cl.build!
    sum_k = cl.Kernel(sum_p, "sum")

    step_size = 0.001

    for i in 1:1000
        queue(diff_k, length(input), nothing, input_buff, diff_buff, size(input)[1], length(input))
        queue(sum_k, length(input), nothing, input_buff, diff_buff, Float32(step_size))
    end
    r = cl.read(queue, input_buff);
    r = reshape(r, size(input))
    return r
end

In [ ]:
@time do_openCL(initial_cond(true), 2);

Running on the GPU is reasonably quick, considering the size of the system. My system typically takes 35secs.

The big array system has a 5000x5000 array and we loop 1000 times across this, performing a number of summation and multiplication operations on every element.

In [ ]:
@time do_openCL(initial_cond(false), 1);

Running on the CPU is very slow. The big array takes >5 mins on my laptop.

So here I've set a flag to false to run an array which is 25 times smaller. It still takes 15 secs on the CPU.

In [ ]:
(25*15) / 60

In [ ]:
@time plot_result = do_openCL(initial_cond(false), 2);

We can run that small system here on the GPU. We see a decent speed-up relative to CPU, on my system the time is now 2 secs, but on such a small system it is no longer dramatic.

In [ ]:
function diff_step(input, output)
    maxX, maxY = size(input)
    for iter in eachindex(input)
        x, y = ind2sub(input, iter)
        output[iter] = - 4. * input[iter] + input[x, y - 1 == 0 ? maxY : y - 1] + 
        input[x, y + 1 > maxY ? 1 : y + 1] +
        input[x - 1 == 0 ? maxX : x - 1, y] +
        input[x + 1 > maxX ? 1 : x + 1, y]
    end
    return output
end

function sum_step(input, diff, step_size)
    return input + (step_size * diff)
end

function do_naive(input)
    step_size = 0.001

    diff = similar(input)
    for i in 1:1000
        diff = diff_step(input, diff)
        input = sum_step(input, diff, step_size)
    end
    return input
end;

We can implement a *very* naive version here in non-optimised pure Julia, to see if there is much overhead using OpenCL.

In [ ]:
@time naive_result = do_naive(initial_cond(false));

This comes in at 26 secs on my machine. Compared with 2 secs for OpenCL on GPU and 15 secs for OpenCL on CPU.

Let's be clear, this is because we are operating in a regime which is particularly suitable for GPU computing here!

In [ ]:
initial_conditions_array = initial_cond(false)

In [ ]:
extrema(initial_conditions_array)

In [ ]:
size(initial_conditions_array)

In [ ]:
extrema(plot_result)

In [ ]:
size(plot_result)

In [ ]:
initial_conditions_array[initial_conditions_array.>0]

In [ ]:
plot_result[plot_result.>0]

In [ ]:
using PyCall
@pyimport matplotlib.plot as mplot

In [ ]:
mplot.plot(initial_conditions_array);

In [ ]:
mplot.imshow(initial_conditions_array[480:520,480:520])

In [ ]:
mplot.imshow(plot_result[480:520,480:520])

We've focused our plot on the area where something is happening. For this particular process, for only 1000 time steps, we could have simulated a 40x40 area instead of 1000x1000.

## Benchmarking, Timing, Profiling

In [ ]:
@time

In [ ]:
using BenchmarkTools
X, Y = rand(1000,1000), rand(1000,1000);
@benchmark max(abs(X), abs(Y))

## Debugging

In [ ]:
using Gallium